In [1]:
import sys
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import math

# Adjust the path for c2qa and bosonic-qiskit modules
module_path = os.path.abspath(os.path.join('..', '..', '..'))  # Moves three directories up
if module_path not in sys.path:
    sys.path.append(module_path)

# Add the `playground` directory to the Python path
playground_path = Path.cwd().parent.parent  # Move two levels up to `playground`
if str(playground_path) not in sys.path:
    sys.path.append(str(playground_path))


# Qiskit and related imports
import qiskit
from qiskit.quantum_info import state_fidelity, Statevector, SparsePauliOp, Operator
from qiskit.circuit import QuantumCircuit
from qiskit.circuit.library import PauliEvolutionGate, UnitaryGate
from qiskit.synthesis import LieTrotter, SuzukiTrotter
import qutip as qt


# Project-specific imports
import c2qa
import c2qa.util as util



This notebook looks at implementing the 1D Holstein model using hybrid qubit-qumode architechture. 
The one-dimensional version of the Holstein model reads

$$
\hat{H} = - \sum_{\langle i, j \rangle} V f^\dagger_i f_j + \sum_i \omega b^\dagger_i b_i + \sum_i g \omega f^\dagger_i f_i (b^\dagger_i + b_i),
$$

with $V$ the hopping coefficient between the nearest neighbour pair $\langle i, j \rangle$, $\omega$ the vibration frequency, and $g$ the coupling constant. For simplicity, we will first choose a two-site Holstein model with periodic boundary conditions:
$$
\begin{aligned}
\hat{H} &= -V \Bigl( f^\dagger_1 f_2 + f^\dagger_2 f_1 \Bigr) \\
&\quad + \omega \Bigl( b^\dagger_1 b_1 + b^\dagger_2 b_2 \Bigr) \\
&\quad + g\,\omega \Bigl[ f^\dagger_1 f_1 \left( b^\dagger_1 + b_1 \right) + f^\dagger_2 f_2 \left( b^\dagger_2 + b_2 \right) \Bigr].
\end{aligned}
$$
We first note that in this architechture, for e.g. the first term, we will need oscillator-mediated multi-qubit gates, however, for now, we ignore this requirement.

The first term $ -V \Bigl( f^\dagger_1 f_2 + f^\dagger_2 f_1 \Bigr) = -V/2(X_1X_2 = Y_1Y_2)$ corresponds to an fSim gate. We define this first below.



In [2]:
def hopping_term(theta):
    """
    Create an fSim gate defined as U(t)=exp(i Vt/2 (X⊗X+Y⊗Y)).

    Parameters:
        V (float): The coupling constant.
        t (float): time.

    Returns:
        UnitaryGate: The corresponding fSim gate.
    """
    
    U = np.array([
        [1,               0,               0, 0],
        [0,  np.cos(theta), 1j * np.sin(theta), 0],
        [0, 1j * np.sin(theta),  np.cos(theta), 0],
        [0,               0,               0, 1]
    ])
    return UnitaryGate(U, label="fSim")


The $\omega \Bigl( b^\dagger_1 b_1 + b^\dagger_2 b_2 \Bigr)$ term can be implemented at each site individually using a phase space rotation gate $\text{exp}(-i\theta n_i)$, with $\theta = \omega$. Finally, the boson-fermion coupling term $f^\dagger_1 f_1 \left( b^\dagger_1 + b_1 \right) = (Z_1 + 1)(b_1^{\dagger} + b_1)$, which is easy to implement as all terms in the sum commute and each
has a simple implementation; the term proportional to $Z_1$ generates a conditional displacement gate and the remaining term is a dispacement.

quick note about displacements: bosonic qiskit defines $D(\alpha) = \text{exp}(\alpha a^\dagger + \alpha* a)$. So, to implement $ \text{exp}(-i\phi(a^\dagger + a))$, we need to choose $D(-i\phi)$. The same is true for conditional displacements.

In [3]:
def simulate_holstein_model(dt, num_trotter, v, omega, g):
    """
    Simulate the Holstein model for 2 sites using a Trotter decomposition.
    
    Args:
        dt (float): Time step for each Trotter step.
        num_trotter (int): Number of Trotter steps.
        v (float): Hopping strength.
        omega (float): Rotation frequency for the bosonic modes.
        g (float): Coupling strength between fermionic and bosonic modes.
    
    Returns:
        state: The final state of the simulation.
    """
    # Set number of qubits per qumode and number of sites
    num_qubits_per_qumode = 3
    num_sites = 2

    # Create the bosonic and qubit registers
    qmr = c2qa.QumodeRegister(num_qumodes=num_sites, num_qubits_per_qumode=num_qubits_per_qumode)
    qbr = qiskit.QuantumRegister(num_sites)
    circuit = c2qa.CVCircuit(qmr, qbr)


    # Loop over Trotter steps, applying each term scaled by dt.
    for step in range(num_trotter):
        # Hopping term between fermion
        fsim_gate = hopping_term(v * dt/2)
        circuit.append(fsim_gate, [qbr[0], qbr[1]])

        # Bosonic rotation terms (each qumode rotates by omega*dt)
        circuit.cv_r(omega * dt, qmr[0])
        circuit.cv_r(omega * dt, qmr[1])

        # Coupling terms between bosonic modes and fermionic qubits.
        # Here the coupling strength is g*omega/2, so we multiply by dt.
        circuit.cv_c_d(-1j*(g * omega / 2) * dt, qmr[0], qbr[0])
        circuit.cv_d(-1j*(g * omega / 2) * dt, qmr[0])
        circuit.cv_c_d(-1j*(g * omega / 2) * dt, qmr[1], qbr[1])
        circuit.cv_d(-1j*(g * omega / 2) * dt, qmr[1])


    # Run the simulation without noise and return the final state
    state, _, _ = c2qa.util.simulate(circuit)
    return state
    
# # example
# dt = 0.1
# num_trotter = 1
# v = 1.0
# omega = 1.0
# g = 0.5

# # Run the simulation
# final_state = simulate_holstein_model(dt, num_trotter, v, omega, g)
# print(final_state)